In [3]:
import torch
import torch.nn as nn
from MyTransformer import EmbeddingTransformer

In [37]:
class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.EmbeddingTransformer = EmbeddingTransformer(src_vocab_size=21000, d_model=512, num_heads=8, num_layers=6, d_ff=2048, max_seq_length=128)
        self.fc1 = nn.Linear(512, 2)
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        embeddings = self.EmbeddingTransformer(x)
        summed_embeddings = torch.sum(embeddings, dim=1)
        score = self.fc1(summed_embeddings)
        return self.softmax(score)
        

In [8]:
emb_trans = EmbeddingTransformer(src_vocab_size=21000, d_model=512, num_heads=8, num_layers=6, d_ff=2**11, max_seq_length=128)


In [9]:
max_tokens = 128
emb_size = 512

In [31]:
out = emb_trans(torch.randint(0, 21000, (64,max_tokens)))
out.shape

torch.Size([64, 128, 512])

In [38]:
model = Classifier()
rand_input = torch.randint(0, 21000, (64,max_tokens))
out = model(rand_input)
out

tensor([[7.8053e-25, 1.0000e+00],
        [3.9235e-34, 1.0000e+00],
        [2.4307e-27, 1.0000e+00],
        [1.9882e-25, 1.0000e+00],
        [5.6719e-24, 1.0000e+00],
        [2.0365e-17, 1.0000e+00],
        [1.9953e-22, 1.0000e+00],
        [6.2784e-30, 1.0000e+00],
        [1.5836e-26, 1.0000e+00],
        [4.1264e-26, 1.0000e+00],
        [1.2081e-19, 1.0000e+00],
        [2.1368e-24, 1.0000e+00],
        [5.1917e-20, 1.0000e+00],
        [1.5945e-35, 1.0000e+00],
        [4.3090e-32, 1.0000e+00],
        [1.2611e-15, 1.0000e+00],
        [8.4327e-22, 1.0000e+00],
        [5.5823e-21, 1.0000e+00],
        [1.0929e-30, 1.0000e+00],
        [3.7625e-25, 1.0000e+00],
        [6.9743e-19, 1.0000e+00],
        [6.8842e-25, 1.0000e+00],
        [2.3241e-26, 1.0000e+00],
        [1.0085e-15, 1.0000e+00],
        [3.0634e-33, 1.0000e+00],
        [1.0845e-16, 1.0000e+00],
        [1.6879e-24, 1.0000e+00],
        [1.9096e-27, 1.0000e+00],
        [3.0616e-28, 1.0000e+00],
        [2.561